# DATASET

In [8]:
import numpy as np
import h5py
import chess
import chess.pgn
import chess.engine
import chess.svg

In [9]:
# Diccionarios
capa_pieza = {'p': 1, 'r': 3, 'n': 5, 'b': 7, 'q': 9, 'k': 11, 'P': 0, 'R': 2, 'N': 4, 'B': 6, 'Q': 8, 'K': 10}

numero_letra = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

letra_numero = {'a': 0, 'b': 1, 'c': 2, 'd':3 , 'e': 4, 'f': 5, 'g':6 , 'h': 7}


# representación tablero
def representacion_tablero(board,verbose=False):
    fen = board.fen().split()
    tablero_piezas_fen = fen[0]
    turno_fen = fen[1]
    enroque_fen = fen[2]
    enpassant_fen = fen[3]

    tablero_r = np.zeros((8,8,22),dtype=np.float32)

    fila=0
    columna=0
    
    for caracter in tablero_piezas_fen:
        
        if not caracter.isdigit():
         
            if caracter == '/':
                fila+=1
                columna=0
           
            else:
    
                tablero_r[fila][columna][capa_pieza[caracter]]=1
                columna+=1
                
        else:
            columna+= int(caracter)

    if turno_fen == 'w':
        tablero_r[:, :, 12] = 1
    if 'K' in enroque_fen:
        tablero_r[:, :, 13] = 1
    if 'k' in enroque_fen:
        tablero_r[:, :, 14] = 1
    if 'Q' in enroque_fen:
        tablero_r[:, :, 15] = 1
    if 'q' in enroque_fen:
        tablero_r[:, :, 16] = 1
           

    if enpassant_fen != "-":
        tablero_r[8-int(enpassant_fen[1])][letra_numero[enpassant_fen[0]]][17] = 1


    for square in chess.SQUARES:
        row = 7 - (square // 8)
        col = square % 8  
        tablero_r[row, col, 18] = len(board.attackers(chess.WHITE, square))/10
        tablero_r[row, col, 19] = len(board.attackers(chess.BLACK, square))/10

    for move in board.legal_moves:
        to_square = move.to_square
        row = 7 - (to_square // 8)
        col = to_square % 8
        tablero_r[row][col][20] = 1

    if board.is_check():
        tablero_r[:, :, 21] = 1


    if turno_fen == 'b':
        tablero_r = np.flip(tablero_r[:, :, :22], axis=(0, 1)).copy()

    if verbose:       
        for capa in range(22):
             for fila in range(8):
                 for columna in range(8):
                     print(tablero_r[fila][columna][capa], end="")
                     print(" ",end="")
                 print(" ")
             print("\n")



    return tablero_r



# representación movimiento
def move_to_policy(uci_move,turn,verbose=False):
    move = np.zeros((8,8,2),dtype=np.float32)
    casilla_origen = uci_move[:2]
    casilla_destino = uci_move[2:4]

    casilla_origen_columna = letra_numero[casilla_origen[0]]
    casilla_origen_fila = casilla_origen[1]
    
    casilla_destino_columna = letra_numero[casilla_destino[0]]
    casilla_destino_fila = casilla_destino[1]


    casilla_origen_fila = 8 - int(casilla_origen_fila)

    casilla_destino_fila = 8 - int(casilla_destino_fila)


    move[casilla_origen_fila][casilla_origen_columna][0]=1
    move[casilla_destino_fila][casilla_destino_columna][1]=1

    if turn != chess.WHITE:
        move = np.flip(move[:, :, :2], axis=(0, 1)).copy()

    if verbose:
        for capa in range(2):
             for fila in range(8):
                 for columna in range(8):
                     print(move[fila][columna][capa], end="")
                     print(" ",end="")
                 print(" ")
             print("\n")
        
        
    return move

In [12]:
def crear_db(filename_origen,filename_final):
    tiempo_inicial = time.time() 
    with open(filename_origen) as infile:
        with h5py.File(filename_final, 'w') as archivo:
            
            grupo_posiciones = archivo.create_group('posiciones')
            grupo_movimientos = archivo.create_group('movimientos')
            grupo_evaluaciones = archivo.create_group('evaluaciones')
        
            string_pgn=""
        
            n= 0
            for line in infile:
                    
                line = line.strip() 
          
                string_pgn += line + "\n"
                    
                if line.endswith("1/2-1/2") or line.endswith("1-0") or line.endswith("0-1"):
                    
                    
                    if guardar_partida(string_pgn,n,grupo_posiciones,grupo_movimientos,grupo_evaluaciones):
                        n+=1
                       # print(f"Partida {n}  guardada")
                    
                     
                        if n % 1000 == 0:
                            print(f"Partidas guardadas: {n}")
                         

                    string_pgn=""
           
    tiempo_final = time.time()  
    tiempo_transcurrido = tiempo_final - tiempo_inicial  # Calcular el tiempo transcurrido
    
    print("Tiempo transcurrido:", tiempo_transcurrido, "segundos")
            


def guardar_partida(string_pgn,n,grupo_posiciones,grupo_movimientos,grupo_evaluaciones):

    pgn = io.StringIO(string_pgn)
    game = chess.pgn.read_game(pgn)
    board = game.board()  
    resultado = game.headers.get('Result')
    
    move_count = game.end().ply()

    if move_count<255 and move_count>0:
        
        boards = np.zeros((move_count, 8, 8, 22), dtype=np.float32)
        moves = np.zeros((move_count, 8, 8, 2), dtype=np.float32)
        evaluations = np.zeros((move_count), dtype=np.float32)
       
            
        num_posicion = 0
        for move in game.mainline_moves():        

             
            if resultado == "1-0":
                if board.turn==chess.WHITE:
                    eval=1
                else:
                    eval=-1
            elif resultado == "0-1":
                if board.turn==chess.WHITE:
                    eval=-1
                else:
                    eval=1
            else:
                eval = 0
                

            boards[num_posicion] = representacion_tablero(board)
            moves[num_posicion] = move_to_policy(move.uci(),board.turn)
            evaluations[num_posicion] =  eval
 
            board.push(move) 
            num_posicion += 1

          
            
        grupo_posiciones.create_dataset(f'board{n}', data=boards, dtype='float32')
        grupo_movimientos.create_dataset(f'mov{n}', data=moves, dtype='float32')
        grupo_evaluaciones.create_dataset(f'eval{n}', data= evaluations, dtype='float32')
        return True
    else:
        
        return False
            



In [13]:
crear_db("partidas.pgn","dataset.h5")

Partidas guardadas: 1000
Partidas guardadas: 2000
Partidas guardadas: 3000
Partidas guardadas: 4000
Partidas guardadas: 5000
Partidas guardadas: 6000
Partidas guardadas: 7000
Partidas guardadas: 8000
Partidas guardadas: 9000
Partidas guardadas: 10000
Tiempo transcurrido: 382.4198648929596 segundos
